In [1]:
import tensorflow as tf

2022-07-29 03:39:53.331480: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-29 03:39:53.331537: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import requests
import pandas as pd
import numpy as np

In [3]:
global dfEntradas

In [4]:
url = "http://vps-40d69db1.vps.ovh.ca:23004/api/TreinoRedes/obtemListaParaPrevisao"

In [5]:
r = requests.get(url)
j = r.json()
dfTreino = pd.DataFrame.from_dict(j)

In [6]:
dfTreino.head()

,id,grupoAcaoId,periodoTreinoRedeId,tipoExemploTreinoId,regraProjecaoId,redeNeuralId,ativoPrevisao,limiteParaEntrada,tipoCompraVenda
0,8,12,1,1,32,2,1,0.3,C


In [77]:
dia = 20220713
urlExemplos = "http://vps-40d69db1.vps.ovh.ca:23005/api/ExemploTreinoAcaoEntradas/obtemPorDia?diaNum={}".format(dia)

In [78]:
print(urlExemplos)

http://vps-40d69db1.vps.ovh.ca:23005/api/ExemploTreinoAcaoEntradas/obtemPorDia?diaNum=20220713


In [79]:
global dfDados
r = requests.get(urlExemplos)
j = r.json()
dfDados = pd.DataFrame.from_dict(j)

In [80]:
dfDados.head()

,campoX,ticker,diaNumPrevisao
0,"1.059420, 1.052899, 1.048551, 1.047101, 1.048...",ABEV3,20220713
1,"2.264151, 2.249057, 2.256604, 2.249057, 2.249...",AERI3,20220713
2,"1.581432, 1.545124, 1.522303, 1.510892, 1.514...",ALPA4,20220713
3,"1.175864, 1.173438, 1.190418, 1.185567, 1.186...",ALSO3,20220713
4,"1.325893, 1.334821, 1.343750, 1.325893, 1.334...",AMAR3,20220713


In [81]:
def incluiEntrada(treino):
    urlEntrada = "http://vps-40d69db1.vps.ovh.ca:23004/api/CotacaoIntradayAcaoResultados/listaPrecoEntradaPrevisaoPorDiaB3?diaNum=" + str(dia) +"&idRegra=" + str(treino['regraProjecaoId']) + "&idGrupo=" + str(treino['grupoAcaoId'])
    print(urlEntrada)
    r = requests.get(urlEntrada)
    j = r.json()
    dfEntradas = pd.DataFrame.from_dict(j)
    print(dfEntradas.head())
    return dfEntradas

In [82]:
def trataRede(treino):
    print('ID:' , treino['id'])
    nome_arquivo = 'pesos/treino_rede' + str(treino['id']) + '.h5'
    print(nome_arquivo)
    modelo = tf.keras.models.load_model(nome_arquivo)
    modelo.summary()
    result = modelo.predict(X)
    dfDados['resultado'] = result
    dfDados['treinoRedeId'] = str(treino['id'])
    dfDados['tipoCompraVenda'] = treino['tipoCompraVenda']
    dfEntradas = incluiEntrada(treino)
    dfPrevisao = pd.merge(dfDados,dfEntradas, how='left', left_on='ticker' ,  right_on='ticker')
    print('ticker previsão:' , len(dfPrevisao))
    json = dfPrevisao.to_dict(orient='list')
    urlPrevisao = 'http://vps-40d69db1.vps.ovh.ca:23004/api/PrevisaoRedes/recebePrevisaoTreino'
    print(json)
    response = requests.post(urlPrevisao, data=json)
    print(response)

In [83]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')

In [84]:
dfTratadoX = dfDados.apply(trataLinhaX, axis='columns')
X = np.stack(dfTratadoX.values)
X.shape

(87, 1800)

In [85]:
dfDados.drop('campoX', inplace=True, axis=1)

In [86]:
dfTreino.apply(trataRede, axis='columns')

ID: 8
pesos/treino_rede8.h5
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1800)              3241800   
                                                                 
 dense_1 (Dense)             (None, 1400)              2521400   
                                                                 
 dense_2 (Dense)             (None, 1000)              1401000   
                                                                 
 dense_3 (Dense)             (None, 600)               600600    
                                                                 
 dense_4 (Dense)             (None, 200)               120200    
                                                                 
 dense_5 (Dense)             (None, 1)                 201       
                                                                 
Total params: 7,885,201
Trai

0    None
dtype: object